In [1]:
import numpy as np
import torch

from omegaconf import OmegaConf
from deepnote import MusicRepr
from importlib import reload

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

seed_everything(42)

42

## Config

In [2]:
conf = OmegaConf.load('conf.yaml')

## Dataset

In [3]:
from midi_transformer import LMDataset, get_dataloaders


dataset = LMDataset(
    **conf['data']
)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [4]:
len(dataset)

8622281

In [5]:
train_loader, val_loader = get_dataloaders(dataset, batch_size=20, n_jobs=4, val_frac=0.1)

train dataset has 7760053 samples and val dataset has 862228 samples.


In [6]:
x, y = dataset[0]
x.shape, y.shape

((1024, 8), (1024, 8))

In [7]:
for b in val_loader:
    for k in b:
        print(k, b[k].shape)
    break

X torch.Size([20, 1024, 8])
X_len torch.Size([20])
labels torch.Size([20, 1024, 8])


## Model

In [8]:
from midi_transformer import CPTransformer

model = CPTransformer(conf['model'])
print('model has', model.count_parameters(), 'parameters.')

model has 13088005 parameters.


In [9]:
model.step(b)

tensor(3.3427, grad_fn=<DivBackward0>)

## Trainer

In [10]:
name = '-'.join(conf['data']['instruments'])
print('model name:',name)

logger = TensorBoardLogger(save_dir='logs/', name=name)
lr_logger = LearningRateMonitor(logging_interval='step')
checkpoint = ModelCheckpoint(
    dirpath=f'weights/{name}/', 
    filename='{epoch}-{val_loss:.2f}', 
    monitor='val_loss',
    save_top_k=1, 
    period=1
)

trainer = Trainer(
    benchmark=True, 
    gpus=1, 
#     reload_dataloaders_every_epoch=True,
#     gradient_clip_val=0.5,
    accumulate_grad_batches=1,
    logger=logger, 
    max_epochs=conf['model']['max_epochs'],
    callbacks=[checkpoint, lr_logger]
)


/home/soroosh/.local/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Checkpoint directory weights/piano-bass-drums-guitar-ensemble/ exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


model name: piano-bass-drums-guitar-ensemble


In [ ]:
trainer.fit(model, train_loader, val_loader)


  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | emb       | CPEmbedding      | 280 K 
2 | encoder   | Encoder          | 12.6 M
3 | head      | CPHeadLayer      | 190 K 
-----------------------------------------------
13.1 M    Trainable params
0         Non-trainable params
13.1 M    Total params


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
trainer.save_checkpoint(f'weights/{name}/last.ckpt')

## generation

In [ ]:
gen = model.generate(
    prompt=None,
    max_len=100, 
    temperatures={
        'ttype' : 2,
        'position': 0.9,
        'tempo': 0.5,
        'chord': 0.8,
        'instrument': 3.,
        'pitch': 2,
        'duration': 0.8,
        'velocity': 0.8
    }
)

In [ ]:
seq = MusicRepr.from_cp(gen)
seq.to_midi('gen.mid')